In [10]:
import pandas as pd
import numpy as np
import sklearn as sk

In [11]:
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)

In [12]:
train_str = 'data_interpolated_means.csv'
labels_str = 'labels_sorted.csv'

df_train = pd.read_csv(train_str)
df_labels = pd.read_csv(labels_str)

In [21]:
print(df_train.shape)
df_train

(227940, 37)


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,1,34.0,32.893,38.657,12.0,2.349,36.0,8.700,24.000,-2.000,16.00,273.38,4.600,6.3,0.50,45.000,158.23,0.500,143.0,93.271,120.0,84.000,1.20,3.800,61.000,7.6,96.624,100.0,1.172,114.000,24.60,94.0,1.544,6.452,142.000,7.330
1,1,2,34.0,32.893,38.657,12.0,2.349,36.0,8.675,24.250,-2.000,16.00,273.38,4.600,6.1,0.50,44.000,158.23,0.500,143.0,93.271,120.0,81.000,1.35,3.837,62.500,7.6,96.624,100.0,1.172,113.625,24.05,99.0,1.544,6.452,125.000,7.330
2,1,3,34.0,32.893,38.657,12.0,2.349,36.0,8.650,24.500,0.000,18.00,273.38,4.600,5.9,0.50,43.000,158.23,0.400,143.0,93.271,120.0,74.000,1.50,3.875,59.000,7.6,96.624,100.0,1.172,113.250,23.50,92.0,1.544,6.452,110.000,7.370
3,1,4,34.0,32.893,38.657,12.0,2.349,37.0,8.625,24.750,0.000,18.00,273.38,4.600,5.7,0.50,42.857,158.23,0.400,143.0,93.271,120.0,66.000,1.65,3.912,49.500,7.6,96.624,100.0,1.172,112.875,22.95,88.0,1.544,6.452,104.000,7.370
4,1,5,34.0,32.893,38.657,12.0,2.349,37.0,8.600,25.000,0.000,18.00,273.38,4.600,5.5,0.50,42.714,158.23,0.400,143.0,93.271,120.0,63.000,1.80,3.950,48.000,7.6,96.624,100.0,1.172,112.500,22.40,81.0,1.544,6.452,100.000,7.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,31658,8,60.0,32.893,38.657,13.0,2.349,37.0,14.400,23.806,-0.762,18.12,273.38,3.617,4.6,0.71,40.822,158.23,0.658,164.0,93.271,104.0,91.000,1.90,3.600,70.000,8.8,96.624,96.0,1.172,105.922,40.20,71.0,1.544,6.452,127.000,7.377
227936,31658,9,60.0,32.893,38.657,13.0,2.349,37.0,14.400,23.806,-0.762,18.12,273.38,3.617,4.6,0.71,40.822,158.23,0.658,164.0,93.271,106.0,96.667,1.90,3.600,73.667,8.8,96.624,96.0,1.172,105.922,40.20,71.0,1.544,6.452,129.667,7.377
227937,31658,10,60.0,32.893,38.657,13.0,2.349,37.0,14.400,23.806,-0.762,18.12,273.38,3.617,4.6,0.71,40.822,158.23,0.658,164.0,93.271,108.0,102.333,1.90,3.600,77.333,8.8,96.624,96.0,1.172,105.922,40.20,71.0,1.544,6.452,132.333,7.377
227938,31658,11,60.0,32.893,38.657,13.0,2.349,37.0,14.400,23.806,-0.762,18.12,273.38,3.617,4.6,0.71,40.822,158.23,0.658,164.0,93.271,110.0,108.000,1.90,3.600,81.000,8.8,96.624,96.0,1.172,105.922,40.20,71.0,1.544,6.452,135.000,7.377


In [20]:
df_labels.shape

(18995, 16)

In [17]:
TEST1 = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate',
         'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
TEST2 = ['LABEL_Sepsis']
TEST3 = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

df_subtask1 = df_labels[TEST1]
df_subtask2 = df_labels[TEST2]
df_subtask3 = df_labels[TEST3]

In [ ]:
print(12*)

In [ ]:
train_features = df_train_sort_nonNaN.to_numpy()
pids = np.unique(train_features[:,0])
#print(len(pids))
# have to somehow exclude age?

train_features = train_features[:,1:]
train_features = train_features.reshape((len(pids),12*train_features.shape[-1]))
#print(train_features.shape)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
df_train_nopid_time = df_train_sort_nonNaN.drop(columns=['pid','Time'])
#print(df_train_nopid_time)
X = df_train_sort_nonNaN.to_numpy(dtype=float)
#print(X.shape)
y = df_subtask1['LABEL_BaseExcess'].to_numpy(dtype=float)
clf = LinearDiscriminantAnalysis()
#print(y.shape)

In [ ]:
#c = clf.fit(X, y)